### Actions

##### <u>reduce()</u> 

In [58]:
rdd1 = sc.parallelize([1,5,2,6,7,8])
rdd1.getNumPartitions()

2

In [4]:
sumResult = rdd1.reduce(lambda x ,y :x+y)

sumResult

29

#Assumptions
partition 1 - 1,5,6
partition 2 - 2,7,8

Working of reduce - In each partition ,first element in the iterator will be set to x and the <br>function will iterate throgh remaing element one by one and will do the sume.

Working : <br>
partition 1 : x=1 , 1+5=6+6=12 <br>
partition 2 : x=2, 2+7 = 9+8=17 <br>
finally in the driver :
12+17=29

In [6]:
multiResult = rdd1.reduce(lambda x,y:x*y)
multiResult

3360

<b><u>fold()</u></b>

similar to reduce , But additionaly takes a <u>zero value</u> as a parameter.<br>
<u>Zero Value</u>-Its an identity element . 1 for multiplication, 0 for sum and emplty list for concatenation.

In [8]:
sumResult = rdd1.fold(0 ,lambda x,y:x+y)
sumResult

29

In [45]:
rdd1.getNumPartitions()

2

In [9]:
multiResult = rdd1.fold(1,lambda x,y:x*y)
multiResult

3360

In [42]:
multiResult = rdd1.fold("a",lambda x,y:str(x)+str(y))
multiResult

'a251a876a'

In [16]:
rdd1.getNumPartitions()

2

As we can see above 'a' concatenated once in every partition and once at last when the final result gets concatanated in the driver . <br>
count of 'a' = number of partitions + 1

<b>Processing model for reduce and fold </b>

1. Each partition is processed sequentially using a single thread.
2. Partitions are processed in parallel using multiple executors / executor threads.
3. Final merge is performed sequentially using a single thread on the driver.

<b><u>Why to use fold </u><b><br>
    1. handles Empty RDD <br>
    2. Minimizes object creation , so garbage collection is lesser
                    

In [22]:
emptyRDD = sc.parallelize([])
emptyRDD.reduce(lambda x,y:x+y)

ValueError: Can not reduce() empty RDD

In [23]:
emptyRDD.fold(0,lambda x,y:x+y)

0

1. As we can see above reduce() function throwing <u>error</u>, when the RDD is empty.<br>
But fold() function working fine and returning the default value.

In [30]:
testRDD = sc.parallelize([1,25,8,4,2])
#testRDD.getNumPartitions()
testRDD.fold(0,lambda a,b:a+1 )

2

In [40]:
#definition of Spark's fold operation
def fold(self, zeroValue, op):
'''    
Aggregate the elements of each partition, and then the results for all
the partitions, using a given associative function and a neutral "zero
value."
The function C{op(t1, t2)} is allowed to modify C{t1} and return it
as its result value to avoid object allocation; however, it should not
modify C{t2}.
'''
    def func(iterator):
        acc = zeroValue
        for obj in iterator:
            acc = op(obj, acc)
            yield acc
    vals = self.mapPartitions(func).collect()
    return reduce(op, vals, zeroValue)

IndentationError: expected an indented block (<ipython-input-40-078769125f64>, line 10)

#### Few examples with fold

In [47]:
sc.parallelize([1,25,8,4,2], 100).fold(0,lambda a,b:a+1 )

100

In [48]:
sc.parallelize([1,25,8,4,2], 50).fold(0,lambda a,b:a+1 )

50

In [49]:
sc.parallelize([1,25,8,4,2], 1).fold(0,lambda a,b:a+1 )

1

##### to count the number of elements in RDD

In [53]:
rdd1mapped = rdd1.map(lambda x:(x,1))
rdd1mapped.collect()

[(1, 1), (5, 1), (2, 1), (6, 1), (7, 1), (8, 1)]

In [56]:
rdd1mapped.reduce(lambda x,y:(x[0]+y[0],x[1]+y[1]))

(29, 6)

<b>Note</b>: So from the above examples we can see that we should <br>
<u>Use reduce() function to count the total number of elements in a RDD.</u>

In [57]:
rdd1mapped.reduce(lambda x,y:x[1]+y[1])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 301, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 795, in func
    yield reduce(f, iterator, initial)
  File "<ipython-input-57-ec348a327304>", line 1, in <lambda>
TypeError: 'int' object has no attribute '__getitem__'

	at o

Now the Billion dollar question arises that why the above code failed with the error <br><b>"TypeError: 'int' object has no attribute '__getitem__'" </b>, <br>

The answer lies how reduce works , the above code will run siccessfully in each partition and will do the sum and <u>each partition will return Integer.</u> <br>
But after that while doing the final reduce in the driver when it will try to use the same lambda function on the returned integers from the partition , It will fail for the obvious reason.<br>
<b>Learning</b>: In reduce() , Return type of lambda function should be same as that RDD type.